# CH2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 1.2 讀取 .csv

In [2]:
data = pd.read_csv('housing.csv')
data.dropna(inplace=True)
data.info() #data.isnull().any()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20433 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
 9   ocean_proximity     20433 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.7+ MB


In [3]:
x = np.array(data)[:, :8].astype('float32')
y = np.array(data)[:, 8].astype('int')

## Split Data

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test =  train_test_split(x, y, random_state = 1)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((15324, 8), (15324,), (5109, 8), (5109,))

### Preprocessing

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [6]:
scalery = StandardScaler()
y_train = scalery.fit_transform(y_train.reshape(-1, 1))
y_test = scalery.transform(y_test.reshape(-1, 1))

In [7]:
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
  except RuntimeError as e:
    print(e)

keras.backend.clear_session()
np.random.seed(1)
tf.random.set_seed(1)

2023-03-09 15:45:35.119944: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 15:45:35.617042: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/extras/CUPTI/lib64
2023-03-09 15:45:35.617098: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.7/lib64:/usr/local/cuda-11.7/extras/CUPTI/

In [8]:
model = keras.models.Sequential([
    keras.layers.Dense(units=200, use_bias=False,  input_shape=x_train.shape[1:]),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('sigmoid'),
    keras.layers.Dense(units=100, kernel_initializer='he_normal', use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('PReLU'),
    keras.layers.Dense(units=50, kernel_initializer='he_normal',use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('PReLU'),
    keras.layers.Dense(units=1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               1600      
                                                                 
 batch_normalization (BatchN  (None, 200)              800       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               20000     
                                                                 
 batch_normalization_1 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 100)               1

2023-03-09 15:45:36.078128: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 15:45:36.401480: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [9]:
opt = keras.optimizers.SGD(learning_rate=1e-3,clipnorm=0.9)
model.compile(loss='mse', optimizer=opt)

In [10]:
model.fit(x_train, y_train,
          epochs=30,
          validation_split=0.2)

Epoch 1/30


2023-03-09 15:45:38.126244: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-09 15:45:38.129674: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x56207bf554a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-09 15:45:38.129687: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-03-09 15:45:38.159170: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


384/384 [==============================] - 5s 8ms/step - loss: 0.8594 - val_loss: 0.5093
Epoch 2/30
384/384 [==============================] - 1s 3ms/step - loss: 0.4137 - val_loss: 0.3438
Epoch 3/30
384/384 [==============================] - 3s 7ms/step - loss: 0.3814 - val_loss: 0.3189
Epoch 4/30
384/384 [==============================] - 4s 9ms/step - loss: 0.3634 - val_loss: 0.3053
Epoch 5/30
384/384 [==============================] - 2s 6ms/step - loss: 0.3529 - val_loss: 0.2990
Epoch 6/30
384/384 [==============================] - 3s 9ms/step - loss: 0.3441 - val_loss: 0.2881
Epoch 7/30
384/384 [==============================] - 3s 9ms/step - loss: 0.3418 - val_loss: 0.2835
Epoch 8/30
384/384 [==============================] - 2s 6ms/step - loss: 0.3332 - val_loss: 0.2782
Epoch 9/30
384/384 [==============================] - 2s 4ms/step - loss: 0.3258 - val_loss: 0.2750
Epoch 10/30
384/384 [==============================] - 2s 4ms/step - loss: 0.3245 - val_loss: 0.2729
Epoch 11/3

In [11]:
model.evaluate(x_test, y_test)

160/160 [==============================] - 0s 944us/step - loss: 0.2474


0.24739202857017517